<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

ModuleNotFoundError: No module named 'piplite'

In [ ]:
import folium
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [2]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)
df = spacex_df
#print(df.columns)


# assume df is your spacex dataframe
import pandas as pd
import numpy as np

# assume df is your spacex dataframe
df = df.copy()

# quick sanity checks (optional, uncomment if you want to inspect)
# print(df.columns)
# print(df['Booster Version'].unique()[:20])
# print(df[['Launch Site', 'class', 'Payload Mass (kg)', 'Booster Version']].head())

# 1) Site with largest number of successful launches (count)
success_mask = df['class'] == 1
site_success_counts = df[success_mask].groupby('Launch Site').size().sort_values(ascending=False)

if not site_success_counts.empty:
    site_largest_success = site_success_counts.index[0]
    print(f"Site with largest number of successful launches: {site_largest_success} ({site_success_counts.iloc[0]} successful launches)")
else:
    print("No successful launches found in the data.")

# 2) Site with highest success rate (percentage)
site_stats = df.groupby('Launch Site')['class'].agg(total_success='sum', total_launches='count')
if not site_stats.empty:
    site_stats['success_rate'] = site_stats['total_success'] / site_stats['total_launches']
    best_site = site_stats['success_rate'].idxmax()
    best_site_rate = site_stats.loc[best_site, 'success_rate']
    best_site_counts = site_stats.loc[best_site, ['total_success','total_launches']].astype(int).tolist()
    print(f"Site with highest success rate: {best_site} ({best_site_rate:.2%} — {best_site_counts[0]}/{best_site_counts[1]} successes/launches)")
else:
    print("No site data to compute success rates.")

# 3) Payload-range success rates (dynamic bins)
payload_col = 'Payload Mass (kg)'
if payload_col not in df.columns:
    print(f"Column '{payload_col}' not found in dataframe.")
else:
    min_p = df[payload_col].min()
    max_p = df[payload_col].max()

    if pd.isna(min_p) or pd.isna(max_p):
        print("Payload column contains only NaNs.")
    elif min_p == max_p:
        print(f"All payload values are identical ({min_p}); cannot create ranges.")
    else:
        # Make 5 bins (you can change num=6 for different number of bins)
        bins = np.linspace(min_p, max_p, num=6)
        # create readable labels
        labels = [f"{int(round(bins[i]))}–{int(round(bins[i+1]))}" for i in range(len(bins)-1)]
        df['payload_bin'] = pd.cut(df[payload_col], bins=bins, labels=labels, include_lowest=True, right=False)

        payload_stats = df.groupby('payload_bin')['class'].agg(total_success='sum', total_launches='count')
        if payload_stats['total_launches'].sum() == 0:
            print("No launches fall into the generated payload bins.")
        else:
            payload_stats['success_rate'] = payload_stats['total_success'] / payload_stats['total_launches']
            payload_stats = payload_stats.dropna(subset=['success_rate'])
            if payload_stats.empty:
                print("No payload-bin success rates available.")
            else:
                high_bin = payload_stats['success_rate'].idxmax()
                low_bin  = payload_stats['success_rate'].idxmin()
                print(f"Payload range with highest success rate: {high_bin} ({payload_stats.loc[high_bin,'success_rate']:.2%} — "
                      f"{int(payload_stats.loc[high_bin,'total_success'])}/{int(payload_stats.loc[high_bin,'total_launches'])})")
                print(f"Payload range with lowest success rate:  {low_bin} ({payload_stats.loc[low_bin,'success_rate']:.2%} — "
                      f"{int(payload_stats.loc[low_bin,'total_success'])}/{int(payload_stats.loc[low_bin,'total_launches'])})")

# 4) Booster Version with highest success rate (uses 'Booster Version' column)
bv_col = 'Booster Version'
if bv_col not in df.columns:
    print(f"Column '{bv_col}' not found in dataframe.")
else:
    booster_stats = df.groupby(bv_col)['class'].agg(total_success='sum', total_launches='count')
    if booster_stats.empty:
        print("No Booster Version data available to compute rates.")
    else:
        booster_stats['success_rate'] = booster_stats['total_success'] / booster_stats['total_launches']
        # sort for inspection and stable results
        booster_stats = booster_stats.sort_values('success_rate', ascending=False)
        top_booster = booster_stats.index[0]
        rate = booster_stats.loc[top_booster, 'success_rate']
        s_count = int(booster_stats.loc[top_booster, 'total_success'])
        t_count = int(booster_stats.loc[top_booster, 'total_launches'])
        print(f"Booster version with highest success rate: {top_booster} ({rate:.2%} — {s_count}/{t_count})")

        # Optional: print the full booster table for inspection
        print("\nAll booster versions (sorted by success rate):")
        print(booster_stats[['total_success','total_launches','success_rate']].head(10).to_string())




ModuleNotFoundError: No module named 'js'

Now, you can take a look at what are the coordinates for each site.


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [3]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

NameError: name 'folium' is not defined

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [4]:
import folium
from folium.features import DivIcon

# Center the map
nasa_coordinate = [28.583, -80.583]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Add circles and text markers
for index, row in launch_sites_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=500,
        color='blue',
        fill=True,
        fill_color='cyan',
        fill_opacity=0.7,
        popup=row['Launch Site']
    ).add_to(site_map)

    folium.map.Marker(
        [row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12pt; color:#d35400;"><b>{row["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Display map directly in notebook (JupyterLite compatible)
site_map


NameError: name 'launch_sites_df' is not defined

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [ ]:
spacex_df.tail(10)

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [ ]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [ ]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Define a function to assign marker colors
def assign_marker_color(class_value):
    if class_value == 1:
        return 'green'
    else:
        return 'red'

# Apply the function to the 'Class' column
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)

# Display the first few rows to verify
spacex_df.head()

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Add marker_cluster to the map
site_map.add_child(marker_cluster)

# Loop through each launch record
for index, record in spacex_df.iterrows():
    # Create a marker for each launch
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  # launch coordinates
        icon=folium.Icon(color=record['marker_color']),  # success/failure color
        popup=(
            f"Launch Site: {record['Launch Site']}<br>"
            f"Success: {record['class']}"
        )
    )

    # Add the marker to the cluster
    marker_cluster.add_child(marker)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline launt site KSC LC-39A
# e.g.,: Lat: 28.56367  Lon: -80.57163
launch_site_lat = 28.573255
launch_site_lon = -80.646895
coastline_lat = 28.55891
coastline_lon = -80.60059
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(distance_coastline)

In [ ]:
from folium.features import DivIcon

# Coordinates
launch_site_coord = [launch_site_lat, launch_site_lon]   # KSC LC-39A
coastline_coord = [coastline_lat, coastline_lon]        # Closest coastline point

# Create a marker on the coastline showing the distance
distance_marker = folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12pt; color:#d35400;"><b>{}</b></div>'.format(
            "{:10.2f} KM".format(distance_coastline)
        )
    )
)

# Add the marker to the map
distance_marker.add_to(site_map)

# Display the map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Coordinates as a list of two points: launch site and coastline
coordinates = [
    [launch_site_lat, launch_site_lon],  # Launch site
    [coastline_lat, coastline_lon]       # Closest coastline point
]

# Create a PolyLine object
lines = folium.PolyLine(
    locations=coordinates,
    weight=2,          # thickness of the line
    color='blue',      # line color
    opacity=0.7        # transparency
)

# Add the line to the map
site_map.add_child(lines)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [ ]:
from folium.features import DivIcon
import folium

# Assume calculate_distance is defined
# Known coordinates
launch_site_lat = 28.56194
launch_site_lon = -80.57722

city_lat = 28.53917
city_lon = -80.67200

highway_lat = 28.4500
highway_lon = -81.2525

# Calculate distances
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Add city marker + line
city_marker = folium.Marker(
    location=[city_lat, city_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt; color:#d35400;"><b>{distance_city:10.2f} KM</b></div>'
    )
)
city_marker.add_to(site_map)

city_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
    color='blue',
    weight=2,
    opacity=0.7
)
site_map.add_child(city_line)

# Add highway marker + line
highway_marker = folium.Marker(
    location=[highway_lat, highway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt; color:#d35400;"><b>{distance_highway:10.2f} KM</b></div>'
    )
)
highway_marker.add_to(site_map)

highway_line = folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
    color='purple',
    weight=2,
    opacity=0.7
)
site_map.add_child(highway_line)

# Show the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
